In [ ]:
import os
import json
from datetime import datetime
from dotenv import load_dotenv
from typing import List, TypedDict, Annotated
from typing_extensions import TypedDict
from langchain_ollama.llms import OllamaLLM
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers.structured import StructuredOutputParser, ResponseSchema
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, END
from langgraph.graph.message import AnyMessage, add_messages
from database import MongoDB
from langchain_core.runnables import RunnableConfig

def synchronization(state):
    load_dotenv()
    MONGODB_USER_NAME = os.getenv("MONGODB_USER_NAME")
    MONGODB_USER_PASSWORD= os.getenv("MONGODB_USER_PASSWORD")

    db = MongoDB(MONGODB_USER_NAME, MONGODB_USER_PASSWORD)
    org_questions = state["org_questions"]
    expanded_questions = state["list_of_questions"]

    total = org_questions + expanded_questions
    db.add_question(total)
    db.close_connection()
    if len(state["messages"]) > 20:
        state["messages"] = state["messages"][2:]
    return {"messages": state["messages"], "list_of_questions": [], "org_questions": []}


question_data_extractor_graph = StateGraph(QuestionPipeline)
question_data_extractor_graph.add_node("learning_assistant",ml_learner_chain)
question_data_extractor_graph.add_node("question_expander",question_extractor_chain)
question_data_extractor_graph.add_node("database_synchronization", synchronization)

question_data_extractor_graph.set_entry_point("learning_assistant")
question_data_extractor_graph.add_edge("learning_assistant", "question_expander")
question_data_extractor_graph.add_edge("question_expander", "database_synchronization")
question_data_extractor_graph.add_edge("database_synchronization", END)


memory = MemorySaver()

question_data_extractor = question_data_extractor_graph.compile(
    checkpointer = memory,
    debug = True
)



MongoDB connection closed.


In [2]:

def question():
    thread_id = 100
    config = RunnableConfig(
        configurable={
            "thread_id": thread_id,
        },
        recursion_limit=25
    )
    standard_ans = "what questions should i ask if i want to further expand on each question"
    starter = """
I am keen on learning AI and ML but I am a complete beginner. I hired an expert where he will answer any questions that I have.
To extract valuable information from this expert, I need your assistant in making valuable queries that allow me to move from 
beginner to intermidate level and eventually ML specialist.
"""

    iteration = 100
    for i in range(iteration):
        if i == 0:
            question = starter
        else:
            question = standard_ans
        
        events = question_data_extractor.stream({"messages": [HumanMessage(content=question, name="User")]}, config, stream_mode="values")
        for event in events:
            print(event)

In [ ]:
def main():
    try:
        question()
    except:
        main()

main()

RecursionError: maximum recursion depth exceeded